# Wolfram Marlowe Smart Contract Execution - Milestone 3

#### Proof of completion for the Project Catalyst proposal #1100291 

### Index

1. [Executive summary](#executiveSummary)<br>
2. [Marlowe review](#marloweReview)<br>
3. [Oracle constraints](#oracleConstraints)<br>
4. [Demonstration case](#demonstrationCase)<br>
5. [Transaction execution](#transactionExecution)<br>
    5.1 [Initialization](#initialization)<br>
    5.2 [Transaction 1: Marlowe Contract Creation](#transactionOne)<br>
    5.3 [Transaction 2: Marlowe Deposit](#transactionTwo)<br>
    5.4 [Transaction 3: Oracle Updater Deployment](#transactionThree)<br>
    5.5 [Transaction 4: Oracle Transaction ](#transactionFour)<br>
    5.6 [Transaction 5: Notify and Close Marlowe Contract ](#transactionFive)<br>


## 1. Executive summary <a name="executiveSummary"></a>

The notebook shows the complete operation of the Wolfram Marlowe Smart Contract Execution components. The notebook presents a real use case and reviews both Marlowe and the Wolfram Oracle contract and the constraints imposed for its proper operation.

## 2. Marlowe review <a name="marloweReview"></a>

### General ideas
Marlowe is a domain specific language for financial applications. It simplifies the creation of smart contracts making it accessible to everyone. 
A Marlowe contract has the following features:

* Finite state, no loops or recursion
* Termination guaranteed
* Limited lifetime
* Assets locking while executing
* No assets retention on close

### Contract blocks

Create a Marlowe contract is combine a set of smaller contract-like building blocks. The next sections review the blocks necessary to request information to the Wolfram Oracle. 

#### When
The `when` block is a "triggered by action" construct. The structure `When [Case] Timeout Contract` allows the Marlowe contract wait for an input from outside the blockchain where `[Case]` is a sequence of transactions, `Timeout` is the time limit to perform actions from `[Case]` and `Contract` is the continuation in case `Timeout` is reached and no `[Case]` happened. `Case` is also called Input. Three inputs are possible:

* `Deposit`: to add assets to the contract
* `Notification`: to include boolean variables
* `Choice`: to insert numerical information

The Wolfram Oracle uses `Choice` to provide data to the contracts. For more information on other inputs, please refer to the [Marlowe Documentation](https://docs.marlowe.iohk.io/docs/introduction).

![onlyWhen](./img/whenOnly.png)

#### Choice
This is the main interface for a Marlowe contract to acquire numerical information from the outside.  A Choice is identified by  a `choice name` and a `choice owner` while `choice bounds` limits the numbers that can be included. When executing this input, the choice is included in the redeemer for the Marlowe input. The transaction must be signed by the `choice owner`.  If the choice is successful, then the contract proceeds with the continuation stated. A choice transaction doesn't change assets balance in the contract.

![onlyChoice](./img/choiceOnly.png)

### Marlowe tools
There are many options to design, test, deploy and interact with Marlowe contract. The most recommended are

* [Marlowe playground](https://playground.marlowe-lang.org/#/blockly): GUI to design and test contracts
* [Marlowe runtime](https://docs.marlowe.iohk.io/docs/developer-tools/runtime/marlowe-runtime): backend application to manage Marlowe contracts on Cardano, includes a REST-API
* [marlowe-cli](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli): off-chain CLI tool to develop contracts

## 3. Oracle constraints <a name="oracleConstraints"></a>

The Marlowe contract and the Oracle contract must fulfill the following requirements:

### Marlowe contract
When defining a Wolfram Oracle input in the Marlowe contract, follow these steps:

* Instantiate a choice.

* Specify the name of the choice.

* Assign ownership to the Oracle Signing Address.

* Set reasonable boundaries for the input.

* Include an additional input to serve as the continuation of the choice.

The last restriction ensures the Marlowe and Oracle contracts can run simultaneously, while the Marlowe validator prevents payouts in multi-contract transactions to avoid double satisfaction attacks.

![choiceImage](./img/choiceImage.png)


### Oracle contract

When creating a request for the oracle service, the following steps are required:

* Progress the Marlowe contract until the oracle input is needed.

* Deploy a UTXO to the Oracle Contract Address, containing the following elements:

    * Payment for the service.
    
    * A datum with the necessary structure.
    
    ```json
    oracleDatum = {
           "marloweContract" :: TxId,
           "marloweIndex" :: Integer,
           "transactionId" :: TxId,
           "transactionIndex" :: Integer,
           "choiceToSolve" :: BuiltinByteString,
           "dataTag" :: BuiltinByteString,
           "deadline":: POSIXTime,
           "beneficiaryAfterDeadline":: PubKeyHash
    }
    ```

### Oracle information
To use the oracle through their steps, the user must have this information available 

* Oracle Signing Address: `addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr`
* Oracle Contract Address: `addr_test1wrs08u8kuqwnse0ahjdjr9ffq20d9vycwy4ns8re2y8sangc8djyc`
* Oracle Contract Reference Script:

```json
{
    "transactionId":"64546010fe2c9745a48087413a81bedd63e9b9c1c97914d6b39fb0690d812483",
    "index":0
}
```

The `Oracle Contract Reference Script` is required if the Oracle Contract reaches its deadline. For further details about the Wolfram Oracle Contract, please refer to the [Milestone One](https://github.com/WolframBlockchainLabs/MarloweSC-Execution/blob/106df96739fe7dd955bc101e0c58929414768f77/MilestoneOne/WolframOracleContract.ipynb) of this proposal.

## 4. Demonstration case <a name="demonstrationCase"></a>

This demonstration features a simple contract that receives an ADA amount, takes input from the oracle, and splits the initial amount based on the oracle's input.

### Contract design

Using the Marlowe logic blocks, the workflow would be like this. 

* Start of the contract

* User deposits 25 Ada (25000000 lovelace) to their account

* The oracle provides the price of BTC/USDT as a choice

* User inputs a "True" notification

* The contract uses the oracle input to split the 25 ada in two amounts, one with `n` lovelace and the other with `25000000-n` lovelace, both payed to the same address

* Contract closes

In blockly format the contract looks like this:

![blockMarloweContract](./img/marloweContractHD.png)

You can check this contract on the [Marlowe Playground](https://playground.marlowe-lang.org/#/importContract?marlowe-view=editor&contract=N4Ig7gFgpgdiBcBtUAXad6krByRp03HV1RKJQHsBrQ1G2AfRgEMBbKBEEAGhAGMArgCdhsfgE9GAZwlsARpQA2XEAF8%2BVBKAAOLYSgnaQLACamx06VzMXGKKNJQBGAG7ChAVh0BOAGYAzABsAObCEoIsUADsYABeMBAAHhIhngAcYEn%2BkiyCEgCOwoJxKOlQ-AAs6XH8KAWFnipqGvjkAkqU0px8ekZErixKglCMlH6M-BCUAJb8nERTs-NjYDBQwsa2ltbwJubC9o4urgUAVukiphFJ1BIQPp5JAa4ADPeClVB%2BKOaCNfx%2BAEAEyVEJxMABHxBfjA9KuEL8MARMLqPhLOajVgcLiUYQsfhKKAAAgx83UrT8wkobEYBP4lEEMBQWwOjl2%2BzsDicbg8gm8-mCYQiUViCWSqQyWRyEjyhWKpXKVQB9UazVa-BY3WMMEoKBmfikBoQKGKUBaAF1NDMOIyUJNKMyZjBIvrHVxCV0evgbVA7QhnNFgUFXtFoiDnD50tENVqFqA-HjJtNMcYyaNKGsNqyLOybAcjjzThcrjc7g8ni93hBPt9fqZ-rUgaDwZDobD4YjkYJURqUytsQsQHiCUTSf3OH3KF7GPIoCgwFBCHgtPBgZ5Xput3wqTSA2oLZbrbbBPaGU6XSw3RgOl7eD6Tyz4IHg6Ggz4fFCfLHtUQ9AZ%2BlAbY8z2bZCxOPkBUCUJwkiGJ4kSFI0kybI-FyfIihKMoKmqWpVQKJo0WHCYqFoDB6DI5h2CHe8hFEcQpFkBRlFUVpnSoOlAUZZkcx2fMuWOXkvF8aDhTgsVEMlFCZTlTDFRwlUGgI5o%2BFMKAdC6GYUF2dcty3I8Hz9U8HQvV0ZndPZPW1Y8jKfF8QzDOFKlBAI1CAA).

## 5. Transaction execution <a name="transactionExecution"></a>

## Initialization  <a name="initialization"></a>
### Environment and versions
Standard tools from *[Marlowe starter kit](https://github.com/input-output-hk/marlowe-starter-kit/tree/main)*. Notice the use of the "preprod" testnet. 

In [2]:
source ../../../scripts/check-tools-and-env.sh

########################
## Check CLI commands ##
########################

The following required programs are available in the shell:
  * jq
  * json2yaml
  * marlowe-cli
  * marlowe-runtime-cli
  * cardano-cli
  * cardano-address
  * cardano-wallet

#########################
## Check required envs ##
#########################

The following environment variables are available in the shell:
  * CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
  * MARLOWE_RT_HOST = proxy
  * MARLOWE_RT_PORT = 3700
  * MARLOWE_RT_WEBSERVER_HOST = web-server
  * MARLOWE_RT_WEBSERVER_PORT = 3780
  * MARLOWE_RT_WEBSERVER_URL = http://web-server:3780

###################
## Check Network ##
###################

The NETWORK is set to preprod
CARDANO_TESTNET_MAGIC = 1
CARDANO_SCAN_URL = https://preprod.cardanoscan.io
MARLOWE_SCAN_URL = https://preprod.marlowescan.com


The versions used to execute transactions are:

In [2]:
marlowe-cli --version
cardano-cli --version

marlowe-cli 0.2.0.0
cardano-cli 8.1.2 - linux-x86_64 - ghc-8.10
git rev d2d90b48c5577b4412d5c9c9968b55f8ab4b9767


### Utilities

Function to select a UTXO based on a minimun quantity.

In [3]:
function select_utxo {
marlowe-cli util select \
  --testnet-magic 1 \
  --lovelace-only "$1" \
  "$PAYMENT_ADDR" \
| sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p'
}

In [4]:
function select_utxo_oracle {
marlowe-cli util select \
  --testnet-magic 1 \
  --lovelace-only "$1" \
  "$ORACLE_ADDR" \
| sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p'
}

### Credentials

Load and store keys and address for the user in the `PAYMENT_SKEY` and `PAYMENT_ADDR` variables. 

The Oracle Signing Address mentioned in section "Oracle information" is stored in the `ORACLE_SIGN_ADDR`.

In [5]:
#PAYMENT_ADDR is the user address
export KEYS=./credentials
PAYMENT_SKEY=$KEYS/lender.skey
PAYMENT_ADDR=$(cat $KEYS/lender.address)
echo "PAYMENT_ADDR=$PAYMENT_ADDR"

PAYMENT_ADDR=addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l


In [5]:
ORACLE_SIGN_ADDR=addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr
echo "ORACLE_SIGN_ADDR=$ORACLE_SIGN_ADDR"

ORACLE_SIGN_ADDR=addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr


### Constants
Ada to Lovelace equivalence

In [7]:
ADA=1000000

### POSIX Milliseconds and Slots

Constants to calculate validity intervals for Marlowe transactions.

In [8]:
SLOT_LENGTH=$(marlowe-cli util slotting --testnet-magic 1| jq -r .scSlotLength)
SLOT_ZERO_TIME=$(marlowe-cli util slotting --testnet-magic 1| jq -r .scSlotZeroTime)
echo "SLOT_LENGTH = $SLOT_LENGTH"
echo "SLOT_ZERO_TIME = $SLOT_ZERO_TIME"

SLOT_LENGTH = 1000
SLOT_ZERO_TIME = 1655683200000


### Marlowe Validator Address
Storing Marlowe Semantics Validator address and reference script UTXO into variables.

In [10]:
marlowe-cli contract validator \
  --testnet-magic 1 \
  --out-file /dev/null \
  --print-hash

addr_test1wpsz02qpp3245nwkkzyg9wye7je3vlrwg5jqgufjyqkanpqef22lm

Validator hash: "6027a8010c555a4dd6b08882b899f4b3167c6e4524047132202dd984"


In [11]:
MARLOWE_ADDR=$(marlowe-cli contract address --testnet-magic 1 )
echo "MARLOWE_ADDR = $MARLOWE_ADDR"

MARLOWE_ADDR = addr_test1wpsz02qpp3245nwkkzyg9wye7je3vlrwg5jqgufjyqkanpqef22lm


In [12]:
UTXO_MARLOWE_SCRIPT=$(
marlowe-cli transaction find-published \
  --testnet-magic 1 \
  2> /dev/null \
| jq -r .marlowe.txIn
)
echo $UTXO_MARLOWE_SCRIPT

6887579ed3f0e34e6f6949a0827b19b6d0ab7895e4e7f6049985548cc2a76095#1


## Marlowe contract preparation <a name="preparation"></a>

### Initial Contract

Following the design specified before.

```haskell
When
    [Case
        (Deposit
            (Address "addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l")
            (Address "addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l")
            (Token "" "")
            (Constant 25000000)
        )
        (When
            [Case
                (Choice
                    (ChoiceId
                        "oracle choice"
                        (Address "addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr")
                    )
                    [Bound 1 25000000]
                )
                (When
                    [Case
                        (Notify TrueObs )
                        (Pay
                            (Address "addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l")
                            (Party (Address "addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l"))
                            (Token "" "")
                            (ChoiceValue
                                (ChoiceId
                                    "oracle choice"
                                    (Address "addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr")
                                ))
                            Close 
                        )]
                    1726077321987 Close 
                )]
            1726077299399 Close 
        )]
    1726077284243 Close 
```


### Contract
Definition of deadlines for each input in the contract and create the .contract file. Notice that the `choice owner` in line 25 is the `ORACLE_SIGN_ADDR` variable declared above.

In [13]:
PAYMENT_DEADLINE=$((1000 * $(date -d "$(date -u) + 8 days" +%s))) #deadline
CHOICE_DEADLINE=$((1000 * $(date -d "$(date -u) + 9 days" +%s))) #deadline
NOTIFY_DEADLINE=$((1000 * $(date -d "$(date -u) + 10 days" +%s))) #deadline

In [14]:
yaml2json << EOI > tx-1.contract
timeout: $PAYMENT_DEADLINE
timeout_continuation: close
when:
- case:
    deposits: 25000000
    into_account:
      address: $PAYMENT_ADDR
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: $PAYMENT_ADDR
  then:
    timeout: $CHOICE_DEADLINE
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 1
          to: 25000000
        for_choice:
          choice_name: oracle input
          choice_owner:
            address: $ORACLE_SIGN_ADDR
      then:
        timeout: $NOTIFY_DEADLINE
        timeout_continuation: close
        when:
        - case:
            notify_if: true
          then:
            from_account:
              address: $PAYMENT_ADDR
            pay:
              value_of_choice:
                choice_name: oracle input
                choice_owner:
                  address: $ORACLE_ADDR
            then: close
            to:
              party:
                address: $PAYMENT_ADDR
            token:
              currency_symbol: ''
              token_name: ''
EOI
json2yaml tx-1.contract

timeout: 1725145631000
timeout_continuation: close
when:
- case:
    deposits: 25000000
    into_account:
      address: addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l
  then:
    timeout: 1725232031000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 1
          to: 25000000
        for_choice:
          choice_name: oracle input
          choice_owner:
            address: addr_test1vqj8urdyuxkyh95x3v0yhu4eftddu8zcc324gzw396c28vgcwyugr
      then:
        timeout: 1725318431000
        timeout_continuation: close
        when:
        - case:
            notify_if: true
          then:
            from_account:
              address: addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l
            pay:
              value_of_choice:
                choice_name: or

### State

Creation of initial state for the contract.

In [15]:
INITIAL_LOVELACE=$((3 * ADA))

In [16]:
yaml2json << EOI > tx-1.state
accounts:
- - - address: $PAYMENT_ADDR
    - currency_symbol: ''
      token_name: ''
  - $INITIAL_LOVELACE
boundValues: []
choices: []
minTime: 1
EOI
cat tx-1.state

{"accounts":[[[{"address":"addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l"},{"currency_symbol":"","token_name":""}],3000000]],"boundValues":[],"choices":[],"minTime":1}


## Transaction 1: Marlowe Contract Creation <a name="transactionOne"></a>

The transactions are constructed manually. Datums and redeemers will be created using marlowe-cli.

In [17]:
marlowe-cli run initialize \
  --testnet-magic 1 \
  --contract-file tx-1.contract \
  --state-file tx-1.state \
  --out-file tx-1.marlowe

In [18]:
marlowe-cli contract datum \
  --contract-file tx-1.contract \
  --state-file tx-1.state \
  --out-file tx-1.datum

a951e7c73680e7d2dbee6752d043a8a90c02fe15a72871fe91a1595e57ac739e


In [19]:
#we select the first input manually.
TX_1_IN="$(select_utxo $((100 * ADA)))"
echo "TX_1_IN = $TX_1_IN"

TX_1_IN = 0b17eed121c4168a7ded8c66520be823954eeb2f14e465185bb0fdc2f71c6a3e#2


Build and submit the contract creation transaction.

In [20]:
cardano-cli transaction build \
  --babbage-era \
  --testnet-magic 1 \
  --tx-in $TX_1_IN \
  --tx-out "$MARLOWE_ADDR+$INITIAL_LOVELACE" \
    --tx-out-datum-embed-file tx-1.datum \
  --change-address $PAYMENT_ADDR \
  --out-file tx-1.unsigned

Estimated transaction fee: Lovelace 192429


In [21]:
cardano-cli transaction sign \
  --signing-key-file $PAYMENT_SKEY \
  --tx-body-file tx-1.unsigned \
  --out-file tx-1.signed

In [22]:
TX_1=$(cardano-cli transaction txid --tx-file tx-1.signed)
echo "TX_1 = $TX_1"

TX_1 = 03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409


In [23]:
cardano-cli transaction submit \
  --testnet-magic 1 \
  --tx-file tx-1.signed

Transaction successfully submitted.


In [24]:
echo "$CARDANO_SCAN_URL/transaction/$TX_1?tab=summary"

https://preprod.cardanoscan.io/transaction/03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409?tab=summary


## Transaction 2: Marlowe Deposit  <a name="transactionTwo"></a>

Executing the deposit transaction.

In [25]:
DEPOSIT_LOVELACE=$((25 * ADA))

In [26]:
TX_2_INVALID_BEFORE=$((1000 * ($(date -u +%s) - 60)))
TX_2_INVALID_HEREAFTER=$((1000 * ($(date -u +%s) + 300)))
echo "TX_2_INVALID_BEFORE = $TX_2_INVALID_BEFORE"
echo "TX_2_INVALID_HEREAFTER = $TX_2_INVALID_HEREAFTER"

TX_2_INVALID_BEFORE = 1724454451000
TX_2_INVALID_HEREAFTER = 1724454811000


In [27]:
SLOT_INIT=$(((TX_2_INVALID_BEFORE - SLOT_ZERO_TIME) / SLOT_LENGTH))
SLOT_END=$(((TX_2_INVALID_HEREAFTER - SLOT_ZERO_TIME) / SLOT_LENGTH))
echo "SLOT_INIT= $SLOT_INIT"
echo "SLOT_END = $SLOT_END"

SLOT_INIT= 68771251
SLOT_END = 68771611


In [28]:
marlowe-cli run prepare \
  --deposit-account $PAYMENT_ADDR \
  --deposit-party $PAYMENT_ADDR \
  --deposit-amount $DEPOSIT_LOVELACE \
  --invalid-before $TX_2_INVALID_BEFORE \
  --invalid-hereafter $TX_2_INVALID_HEREAFTER \
  --marlowe-file tx-1.marlowe \
  --out-file tx-2.marlowe

Rounding  `TransactionInput` txInterval boundaries to:(POSIXTime {getPOSIXTime = 1724454451000},POSIXTime {getPOSIXTime = 1724454811999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1724454451000},POSIXTime {getPOSIXTime = 1724454811999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l\"" "\"addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l\"" (Token "" "") 25000000)]}


In [29]:
jq '.tx.inputs[0]' tx-2.marlowe > tx-2.inputs
marlowe-cli contract redeemer \
  --input-file tx-2.inputs \
  --out-file tx-2.redeemer

In [30]:
jq .tx.contract tx-2.marlowe > tx-2.contract
jq .tx.state tx-2.marlowe > tx-2.state
marlowe-cli contract datum \
  --contract-file tx-2.contract \
  --state-file tx-2.state \
  --out-file tx-2.datum

8da608b410358a1f5f0a5c77f4d8d5508cd96a6d4417aa185e549ae4d71f196c


In [31]:
DEPOSIT_INPUT="$TX_1#1"
echo "DEPOSIT_INPUT = $DEPOSIT_INPUT"

DEPOSIT_INPUT = 03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409#1


Build and submit the deposit transaction.

In [32]:
cardano-cli transaction build \
  --babbage-era \
  --testnet-magic 1 \
  --tx-in-collateral "$DEPOSIT_INPUT" \
  --tx-in "$DEPOSIT_INPUT" \
  --tx-in "$TX_1#0" \
    --spending-tx-in-reference "$UTXO_MARLOWE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file tx-1.datum \
    --spending-reference-tx-in-redeemer-file tx-2.redeemer \
  --tx-out "$MARLOWE_ADDR+$((INITIAL_LOVELACE+DEPOSIT_LOVELACE))" \
    --tx-out-datum-embed-file tx-2.datum \
  --change-address $PAYMENT_ADDR \
  --required-signer $PAYMENT_SKEY \
  --invalid-before $(((TX_2_INVALID_BEFORE - SLOT_ZERO_TIME) / SLOT_LENGTH)) \
  --invalid-hereafter $(((TX_2_INVALID_HEREAFTER - SLOT_ZERO_TIME) / SLOT_LENGTH)) \
  --out-file tx-2.unsigned

Estimated transaction fee: Lovelace 509095


In [33]:
cardano-cli transaction sign \
  --signing-key-file $PAYMENT_SKEY \
  --tx-body-file tx-2.unsigned \
  --out-file tx-2.signed

In [34]:
## this is the transaction that must be solved
TX_2=$(cardano-cli transaction txid --tx-file tx-2.signed)
echo "TX_2 = $TX_2"

TX_2 = cce4dda792104ab84bee1a68c799729bcaf634187994784e54a5f2055acf9c88


In [35]:
cardano-cli transaction submit \
  --testnet-magic 1 \
  --tx-file tx-2.signed

Transaction successfully submitted.


In [36]:
echo "$CARDANO_SCAN_URL/transaction/$TX_2?tab=summary"

https://preprod.cardanoscan.io/transaction/cce4dda792104ab84bee1a68c799729bcaf634187994784e54a5f2055acf9c88?tab=summary


## Transaction 3: Oracle Updater Deployment  <a name="transactionThree"></a>

After the deposit is made, the contract awaits the oracle choice. This section shows the steps to perform the request.

### Oracle Contract Address and Payment

Store Oracle Contract address and define how much ADA it will hold as payment for the service.

In [9]:
ORACLE_CONTRACT_ADDR=addr_test1wrs08u8kuqwnse0ahjdjr9ffq20d9vycwy4ns8re2y8sangc8djyc
echo "ORACLE_CONTRACT_ADDR=$ORACLE_CONTRACT_ADDR"

ORACLE_CONTRACT_ADDR=addr_test1wrs08u8kuqwnse0ahjdjr9ffq20d9vycwy4ns8re2y8sangc8djyc


In [38]:
ORACLE_PAYMENT=$((5 * ADA))
echo "ORACLE_PAYMENT=$ORACLE_PAYMENT"

ORACLE_PAYMENT=5000000


### Create "oracle.datum" File  

In [39]:
MRLW_ID=$TX_1
MRLW_IDX=0
echo "MRLW_ID=$MRLW_ID"
echo "MRLW_IDX=$MRLW_IDX"

MRLW_ID=03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409
MRLW_IDX=0


In [40]:
TX_ID=$TX_2 #tx to be solved
TX_IDX=0    #index to be solved
ORACLE_DL=$((1000 * $(date -d "$(date -u) + 90 minutes" +%s))) #deadline to apply choice before refund
echo "TX_ID=$TX_ID"
echo "TX_IDX=$TX_IDX"
echo "ORACLE_DL=$ORACLE_DL"

TX_ID=cce4dda792104ab84bee1a68c799729bcaf634187994784e54a5f2055acf9c88
TX_IDX=0
ORACLE_DL=1724459986000


In [41]:
yaml2json << EOI > wolframOracleUpdater.datum
constructor: 0
fields:
- constructor: 0
  fields:
  - bytes: "$MRLW_ID"
- int: $MRLW_IDX
- constructor: 0
  fields:
  - bytes: "$TX_ID"
- int: $TX_IDX
- bytes: "6f7261636c6520696e707574" #oracle input
- bytes: "4254432F55534454" #"414441555344"             #BTC/USDT #adausd "(ADAUSD + ADAPEN + ADACYN)/3"
- int: $ORACLE_DL
- bytes: "f1ca04a98e903273b9f3853b9888a1dc62a704ef0801f04b3e71538b"
EOI
cat wolframOracleUpdater.datum

{"constructor":0,"fields":[{"constructor":0,"fields":[{"bytes":"03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409"}]},{"int":0},{"constructor":0,"fields":[{"bytes":"cce4dda792104ab84bee1a68c799729bcaf634187994784e54a5f2055acf9c88"}]},{"int":0},{"bytes":"6f7261636c6520696e707574"},{"bytes":"4254432F55534454"},{"int":1724459986000},{"bytes":"f1ca04a98e903273b9f3853b9888a1dc62a704ef0801f04b3e71538b"}]}


### Build Transaction

Build and submit the Oracle Contract creation transaction.

In [42]:
ORACLE_INPUT="$(select_utxo $((5 * ADA)))"
echo $ORACLE_INPUT

0b17eed121c4168a7ded8c66520be823954eeb2f14e465185bb0fdc2f71c6a3e#0


In [43]:
cardano-cli transaction build \
  --babbage-era \
  --testnet-magic 1 \
  --tx-in "$ORACLE_INPUT" \
  --tx-out "$ORACLE_CONTRACT_ADDR+$ORACLE_PAYMENT" \
    --tx-out-datum-embed-file wolframOracleUpdater.datum \
  --change-address $PAYMENT_ADDR \
  --out-file tx-3.unsigned

Estimated transaction fee: Lovelace 175269


In [44]:
cardano-cli transaction sign \
  --signing-key-file $PAYMENT_SKEY \
  --tx-body-file tx-3.unsigned \
  --out-file tx-3.signed

In [45]:
TX_3=$(cardano-cli transaction txid --tx-file tx-3.signed)
echo "TX_3 = $TX_3"

TX_3 = e12e1bf1ca617ba547fae01df059c3e96cf9ac6265887c1d5c49797599a22234


In [46]:
cardano-cli transaction submit \
  --testnet-magic 1 \
  --tx-file tx-3.signed

Transaction successfully submitted.


In [47]:
echo "$CARDANO_SCAN_URL/transaction/$TX_3?tab=summary"

https://preprod.cardanoscan.io/transaction/e12e1bf1ca617ba547fae01df059c3e96cf9ac6265887c1d5c49797599a22234?tab=summary


## Transaction 4: Oracle Transaction

The Wolfram Oracle Harvester executes this transaction automatically. The variable `TX_4` stores the Transaction ID of the Oracle transaction for further use. The transaction can be checked in the Cardano Scan.

In [3]:
# store Oracle transactionID
TX_4=780e2b30dc53b1d7b6e12d3f35f4758e446f9a8da25871eb190c39d238038530
echo "$CARDANO_SCAN_URL/transaction/$TX_4?tab=summary"

https://preprod.cardanoscan.io/transaction/780e2b30dc53b1d7b6e12d3f35f4758e446f9a8da25871eb190c39d238038530?tab=summary


## Transaction 5: Notify and Close Marlowe Contract  <a name="transactionFive"></a>

Closure of Marlowe contract with a `Notify` input.

In [49]:
CHOICE=$((15 * ADA))
echo "CHOICE = $CHOICE"

CHOICE = 15000000


In [50]:
TX_5_INVALID_BEFORE=$((1000 * ($(date -u +%s) - 60)))
TX_5_INVALID_HEREAFTER=$((1000 * ($(date -u +%s) + 300)))
echo "TX_5_INVALID_BEFORE = $TX_5_INVALID_BEFORE"
echo "TX_5_INVALID_HEREAFTER = $TX_5_INVALID_HEREAFTER"

TX_5_INVALID_BEFORE = 1724455455000
TX_5_INVALID_HEREAFTER = 1724455815000


Since the transaction is built manually, the variables `CHOICE_FILE` and `DATUM_FILE` points to the files created by the oracle. If the transaction is constructed using the Marlowe Runtime backend, this step is not necessary.

In [51]:
CHOICE_FILE="../../../Delphos/$TX_1#0/tx-next.marlowe"
echo $CHOICE_FILE
DATUM_FILE="../../../Delphos/$TX_1#0/marloweOutput.datum"
echo $DATUM_FILE

../../../Delphos/03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409#0/tx-next.marlowe
../../../Delphos/03ada6af2b4644d1c59f5fbfb899869f2a4fc8eea6381f6609c9ceeb7e9c6409#0/marloweOutput.datum


In [52]:
marlowe-cli run prepare \
  --notify \
  --invalid-before $TX_5_INVALID_BEFORE \
  --invalid-hereafter $TX_5_INVALID_HEREAFTER \
  --marlowe-file $CHOICE_FILE \
  --out-file tx-5.marlowe

Rounding  `TransactionInput` txInterval boundaries to:(POSIXTime {getPOSIXTime = 1724455455000},POSIXTime {getPOSIXTime = 1724455815999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1724455455000},POSIXTime {getPOSIXTime = 1724455815999}), txInputs = [NormalInput INotify]}


In [53]:
jq '.tx.inputs[0]' tx-5.marlowe > tx-5.inputs
marlowe-cli contract redeemer \
  --input-file tx-5.inputs \
  --out-file tx-5.redeemer

In [54]:
jq .tx.contract tx-5.marlowe > tx-5.contract
jq .tx.state tx-5.marlowe > tx-5.state
marlowe-cli contract datum \
  --contract-file tx-5.contract \
  --state-file tx-5.state \
  --out-file tx-5.datum

00d3b5410b431342b657642504cd65f08fe9f7ec396aac75eb25d812354bbb12


In [55]:
NOTIFY_INPUT="$(select_utxo $((100 * ADA)))"
echo "NOTIFY_INPUT=$NOTIFY_INPUT"

NOTIFY_INPUT=cce4dda792104ab84bee1a68c799729bcaf634187994784e54a5f2055acf9c88#1


In [56]:
echo "$PAYMENT_ADDR+$((INITIAL_LOVELACE+DEPOSIT_LOVELACE-CHOICE))"

addr_test1vrcu5p9f36gryuae7wznhxyg58wx9fcyauyqruzt8ec48zctqyq5l+13000000


In [57]:
cardano-cli transaction build \
  --babbage-era \
  --testnet-magic 1 \
  --tx-in-collateral "$NOTIFY_INPUT" \
  --tx-in "$NOTIFY_INPUT" \
  --tx-in "$TX_4#0" \
    --spending-tx-in-reference "$UTXO_MARLOWE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file $DATUM_FILE \
    --spending-reference-tx-in-redeemer-file tx-5.redeemer \
  --tx-out "$PAYMENT_ADDR+$CHOICE" \
  --tx-out "$PAYMENT_ADDR+$((INITIAL_LOVELACE+DEPOSIT_LOVELACE-CHOICE))" \
  --change-address $PAYMENT_ADDR \
  --required-signer $PAYMENT_SKEY \
  --invalid-before $(((TX_5_INVALID_BEFORE - SLOT_ZERO_TIME) / SLOT_LENGTH)) \
  --invalid-hereafter $(((TX_5_INVALID_HEREAFTER - SLOT_ZERO_TIME) / SLOT_LENGTH)) \
  --out-file tx-5.unsigned

Estimated transaction fee: Lovelace 455459


In [58]:
cardano-cli transaction sign \
  --signing-key-file $PAYMENT_SKEY \
  --tx-body-file tx-5.unsigned \
  --out-file tx-5.signed

In [59]:
TX_5=$(cardano-cli transaction txid --tx-file tx-5.signed)
echo "TX_5 = $TX_5"

TX_5 = 8d0f1202d7a667cf04ccf1da83463c4461f8bd0bca41656cb59c5a116c916219


In [60]:
cardano-cli transaction submit \
  --testnet-magic 1 \
  --tx-file tx-5.signed

Transaction successfully submitted.


In [61]:
echo "$CARDANO_SCAN_URL/transaction/$TX_5?tab=summary"

https://preprod.cardanoscan.io/transaction/8d0f1202d7a667cf04ccf1da83463c4461f8bd0bca41656cb59c5a116c916219?tab=summary
